# Wildfire Risk - AutoML
__Team 3 - Dave Friesen, John Chen, and Kyle Dalope__<br>
__ADS-508-02-SP23__<br><br>
__GitHub link: https://github.com/davefriesen/wildfire-risk__

In [82]:
__authors__ = ['Dave Friesen', 'John Chen', 'Kyle Dalope']
__contact__ = ['dfriesen@sandiego.edu', 'johnchen@sandiego.edu', 'kdalope@sandiego.edu']
__date__ = '2023-03-20'
__license__ = 'MIT'
__version__ = '1.0.1'

# Checking Pre-Requisites From Previous Notebook

In [83]:
%store -r autopilot_train_s3_wf

In [84]:
try:
    autopilot_train_s3_wf
    print("[OK]")
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

[OK]


In [85]:
if not autopilot_train_s3_wf:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] PLEASE RUN THE PREVIOUS 01_PREPARE_DATASET_AUTOPILOT NOTEBOOK.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [86]:
import boto3
import sagemaker
import pandas as pd
import json

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [87]:
#Training

!aws s3 ls $autopilot_train_s3_wf

print(autopilot_train_s3_wf)

2023-03-30 03:34:37    1045513 wildfire_risk_autopilot.csv
s3://sagemaker-us-east-1-857283526476/data/wildfire_risk_autopilot.csv


In [88]:
!aws s3 ls $autopilot_train_s3_wf

2023-03-30 03:34:37    1045513 wildfire_risk_autopilot.csv


In [89]:
import csv

!aws s3 cp $autopilot_train_s3_wf ./tmp/

df = pd.read_csv("./tmp/wildfire_risk_autopilot.csv")
df.head()

download: s3://sagemaker-us-east-1-857283526476/data/wildfire_risk_autopilot.csv to tmp/wildfire_risk_autopilot.csv


,fire,ContainmentDateTime,ControlDateTime,DiscoveryAcres,FireCause,FireDiscoveryDateTime,FireOutDateTime,POOState,InitialLatitude,InitialLongitude,GACC
0,1.0,NaN,NaN,0.1,Unknown,2019/07/05 03:30:08+00,NaN,US-CA,34.409520,-118.457020,OSCC
1,1.0,NaN,NaN,0.1,Unknown,2019/09/01 19:05:06+00,NaN,US-CA,37.064037,-119.508731,OSCC
2,1.0,2018/02/18 21:00:00+00,2018/02/18 21:04:59+00,0.1,Human,2018/02/18 19:38:00+00,2018/10/12 18:59:59+00,US-CA,37.783830,-119.072300,OSCC
3,1.0,NaN,NaN,0.1,Unknown,2018/08/11 12:37:48+00,NaN,US-CA,34.151110,-118.693890,OSCC
4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Setup the S3 Location for the Autopilot-Generated Assets 
This include Jupyter Notebooks (Analysis), Python Scripts (Feature Engineering), and Trained Models.

In [90]:
prefix_model_output = "models/autopilot"

model_output_s3_wf = "s3://{}/{}".format(bucket, prefix_model_output)

print(model_output_s3_wf)

s3://sagemaker-us-east-1-857283526476/models/autopilot


In [91]:
max_candidates = 3

job_config = {
    "CompletionCriteria": {
        "MaxRuntimePerTrainingJobInSeconds": 900,
        "MaxCandidates": max_candidates,
        "MaxAutoMLJobRuntimeInSeconds": 5400,
    },
}

input_data_config = [
    {
        "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": "{}".format(autopilot_train_s3_wf)}},
        "TargetAttributeName": "fire",
    }
]

output_data_config = {"S3OutputPath": "{}".format(model_output_s3_wf)}

In [92]:
#Check for existing autopilot jobs

existing_jobs_response = sm.list_auto_ml_jobs()

In [93]:
num_existing_jobs = 0
running_jobs = 0

if "AutoMLJobSummaries" in existing_jobs_response.keys():
    job_list = existing_jobs_response["AutoMLJobSummaries"]
    num_existing_jobs = len(job_list)
    # print('[INFO] You already created {} Autopilot job(s) in this account.'.format(num_existing_jobs))
    for j in job_list:
        if "AutoMLJobStatus" in j.keys():
            if j["AutoMLJobStatus"] == "InProgress":
                running_jobs = running_jobs + 1
    print("[INFO] You have {} Autopilot job(s) currently running << Should be 0 jobs.".format(running_jobs))
else:
    print("[OK] Please continue.")

[INFO] You have 0 Autopilot job(s) currently running << Should be 0 jobs.


In [94]:
#Launch Autopilot job

from time import gmtime, strftime, sleep

In [95]:
%store -r auto_ml_job_name

try:
    auto_ml_job_name
except NameError:
    timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
    auto_ml_job_name = "automl-dm-" + timestamp_suffix
    print("Created AutoMLJobName: " + auto_ml_job_name)

In [96]:
print(auto_ml_job_name)

automl-dm-30-03-35-20


In [97]:
%store auto_ml_job_name

Stored 'auto_ml_job_name' (str)


In [98]:
max_running_jobs = 1

if running_jobs < max_running_jobs:  # Limiting to max. 1 Jobs
    try:
        sm.create_auto_ml_job(
            AutoMLJobName=auto_ml_job_name,
            InputDataConfig=input_data_config,
            OutputDataConfig=output_data_config,
            AutoMLJobConfig=job_config,
            RoleArn=role,
        )
        print("[OK] Autopilot Job {} created.".format(auto_ml_job_name))
        running_jobs = running_jobs + 1
    except:
        print(
            "[INFO] You have already launched an Autopilot job. Please continue see the output of this job.".format(
                running_jobs
            )
        )
else:
    print(
        "[INFO] You have already launched {} Autopilot running job(s). Please continue see the output of the running job.".format(
            running_jobs
        )
    )

[INFO] You have already launched an Autopilot job. Please continue see the output of this job.


# Analyzing Data and Generate Notebooks

In [99]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while (
    "AutoMLJobStatus" not in job_description_response.keys()
    and "AutoMLJobSecondaryStatus" not in job_description_response.keys()
):
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job has not yet started. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for Autopilot Job to start...")
    sleep(15)

print("[OK] AutoMLJob started.")

[OK] AutoMLJob started.


# Review the SageMaker `Processing Jobs`
* First Processing Job (Data Splitter) checks the data sanity, performs stratified shuffling and splits the data into training and validation. 
* Second Processing Job (Candidate Generator) first streams through the data to compute statistics for the dataset. Then, uses these statistics to identify the problem type, and possible types of every column-predictor: numeric, categorical, natural language, etc.

In [100]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">Processing Jobs</a></b>'.format(
            region
        )
    )
)

In [101]:
%%time

job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]

if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("Starting", "AnalyzingData"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Data analysis phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

[OK] Data analysis phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:857283526476:automl-job/automl-dm-30-03-35-20",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-dm-30-03-35-20",
    "AutoMLJobSeconda

# Generate Notebooks

* Data Exploration
* Candidate Definition

In [102]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while "AutoMLJobArtifacts" not in job_description_response.keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job has not yet generated the artifacts. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for AutoMLJobArtifacts...")
    sleep(15)

print("[OK] AutoMLJobArtifacts generated.")

[OK] AutoMLJobArtifacts generated.


In [103]:
job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

while "DataExplorationNotebookLocation" not in job_description_response["AutoMLJobArtifacts"].keys():
    job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job has not yet generated the notebooks. Please wait. ")
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print("[INFO] Waiting for DataExplorationNotebookLocation...")
    sleep(15)

print("[OK] DataExplorationNotebookLocation found.")

[OK] DataExplorationNotebookLocation found.


In [104]:
generated_resources = job_description_response["AutoMLJobArtifacts"]["DataExplorationNotebookLocation"]
download_path = generated_resources.rsplit("/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb")[0]
job_id = download_path.rsplit("/", 1)[-1]

In [105]:
from IPython.core.display import display, HTML

if not job_id:
    print("No AutoMLJobArtifacts found.")
else:
    display(
        HTML(
            '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/{}/sagemaker-automl-candidates/{}/">S3 Generated Resources</a></b>'.format(
                bucket, prefix_model_output, auto_ml_job_name, job_id
            )
        )
    )

In [106]:
#Download Notebooks and Code

print(download_path)

s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395


In [107]:
try:
    !aws s3 cp --recursive $download_path .
except:
    print('Could not download the generated resources. Make sure the path is correct.')

download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/generated_module/README.md to generated_module/README.md
download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/generated_module/MANIFEST.in to generated_module/MANIFEST.in
download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/generated_module/candidate_data_processors/dpp0.py to generated_module/candidate_data_processors/dpp0.py
download: s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/generated_module/candidate_data_processors/dpp1.py to generat

# Feature Engineering

In [108]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(
            region
        )
    )
)

In [109]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/">Batch Transform Jobs</a></b>'.format(
            region
        )
    )
)

In [110]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
print(job_status)
print(job_sec_status)
if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("FeatureEngineering"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Feature engineering phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

Completed
Completed
[OK] Feature engineering phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:857283526476:automl-job/automl-dm-30-03-35-20",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-dm-30-03-35-

# Model Training and Tuning

In [111]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/">Hyperparameter Tuning Jobs</a></b>'.format(
            region
        )
    )
)

In [112]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/">Training Jobs</a></b>'.format(
            region
        )
    )
)

In [113]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response["AutoMLJobStatus"]
job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
print(job_status)
print(job_sec_status)
if job_status not in ("Stopped", "Failed"):
    while job_status in ("InProgress") and job_sec_status in ("ModelTuning"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response["AutoMLJobStatus"]
        job_sec_status = job_description_response["AutoMLJobSecondaryStatus"]
        print(job_status, job_sec_status)
        sleep(15)
    print("[OK] Model tuning phase completed.\n")

print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

Completed
Completed
[OK] Model tuning phase completed.

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:857283526476:automl-job/automl-dm-30-03-35-20",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-dm-30-03-35-20",
  

# Status for completion

In [114]:
%%time

job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job_description_response["AutoMLJobStatus"]
print(job_status)
if job_status not in ("Stopped", "Failed"):
    while job_status not in ("Completed"):
        job_description_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job_description_response["AutoMLJobStatus"]
        print(job_status)
        sleep(10)
    print("[OK] Autopilot Job completed.\n")
else:
    print(job_status)

Completed
[OK] Autopilot Job completed.

CPU times: user 5.06 ms, sys: 192 µs, total: 5.25 ms
Wall time: 204 ms


# View All Candidates 

In [115]:
candidates_response = sm.list_candidates_for_auto_ml_job(
    AutoMLJobName=auto_ml_job_name, SortBy="FinalObjectiveMetricValue"
)

In [116]:
print(candidates_response.keys())

dict_keys(['Candidates', 'ResponseMetadata'])


In [117]:
while "Candidates" not in candidates_response.keys():
    candidates_response = sm.list_candidates_for_auto_ml_job(
        AutoMLJobName=auto_ml_job_name, SortBy="FinalObjectiveMetricValue"
    )
    print("[INFO] Autopilot Job is generating the Candidates. Please wait.")
    print(json.dumps(candidates_response, indent=4, sort_keys=True, default=str))
    sleep(10)

candidates = candidates_response["Candidates"]
print("[OK] Candidates generated.")

[OK] Candidates generated.


In [118]:
print(candidates[0].keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime', 'CandidateProperties'])


In [119]:
while "CandidateName" not in candidates[0]:
    candidates_response = sm.list_candidates_for_auto_ml_job(
        AutoMLJobName=auto_ml_job_name, SortBy="FinalObjectiveMetricValue"
    )
    candidates = candidates_response["Candidates"]
    print("[INFO] Autopilot Job is generating CandidateName. Please wait. ")
    print(json.dumps(candidates, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] CandidateName generated.")

[OK] CandidateName generated.


In [120]:
while "FinalAutoMLJobObjectiveMetric" not in candidates[0]:
    candidates_response = sm.list_candidates_for_auto_ml_job(
        AutoMLJobName=auto_ml_job_name, SortBy="FinalObjectiveMetricValue"
    )
    candidates = candidates_response["Candidates"]
    print("[INFO] Autopilot Job is generating FinalAutoMLJobObjectiveMetric. Please wait. ")
    print(json.dumps(candidates, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] FinalAutoMLJobObjectiveMetric generated.")

[OK] FinalAutoMLJobObjectiveMetric generated.


In [121]:
print(json.dumps(candidates, indent=4, sort_keys=True, default=str))

[
    {
        "CandidateName": "automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc",
        "CandidateProperties": {
            "CandidateArtifactLocations": {
                "Explainability": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/documentation/explainability/output",
                "ModelInsights": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/documentation/model_monitor/output"
            },
            "CandidateMetrics": [
                {
                    "MetricName": "F1macro",
                    "Set": "Validation",
                    "StandardMetricName": "F1macro",
                    "Value": 0.47442999482154846
                },
                {
                    "MetricName": "PrecisionMacro",
                    "Set": "Validation",
                    "StandardMetricName": "PrecisionMacro",
                    "Value": 0.4722200036048889
                },
                {
              

In [122]:
for index, candidate in enumerate(candidates):
    print(
        str(index)
        + "  "
        + candidate["CandidateName"]
        + "  "
        + str(candidate["FinalAutoMLJobObjectiveMetric"]["Value"])
    )

0  automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc  0.4800800085067749
1  automl-dm-30-03-35-206UAvnGVe6yJ-003-90c9bfc1  0.28891998529434204
2  automl-dm-30-03-35-206UAvnGVe6yJ-001-c232ef5d  0.28641000390052795


# Inspect Trials using Experiments API

In [123]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=auto_ml_job_name + "-aws-auto-ml-job",
)

df = exp.dataframe()
print(df)

                                  TrialComponentName  \
0  automl-dm-30-03-35-206UAvnGVe6yJ-001-c232ef5d-...   
1  automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc-...   
2  automl-dm-30-03-35-206UAvnGVe6yJ-003-90c9bfc1-...   
3  automl-dm-30-03-35-20-dpp1-csv-1-421e5439dce54...   
4  automl-dm-30-03-35-20-dpp2-rpb-1-b8add5e432b44...   
5  automl-dm-30-03-35-20-dpp1-1-e53e30ad081f4f4e9...   
6  automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91a...   
7  automl-dm-30-03-35-20-db-1-48e7c029b9654b7a86c...   

                                         DisplayName  \
0  automl-dm-30-03-35-206UAvnGVe6yJ-001-c232ef5d-...   
1  automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc-...   
2  automl-dm-30-03-35-206UAvnGVe6yJ-003-90c9bfc1-...   
3  automl-dm-30-03-35-20-dpp1-csv-1-421e5439dce54...   
4  automl-dm-30-03-35-20-dpp2-rpb-1-b8add5e432b44...   
5  automl-dm-30-03-35-20-dpp1-1-e53e30ad081f4f4e9...   
6  automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91a...   
7  automl-dm-30-03-35-20-db-1-48e7c029b9654b7a8

# Explore Best Candidate

In [124]:
best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)

In [125]:
print(best_candidate_response.keys())

dict_keys(['AutoMLJobName', 'AutoMLJobArn', 'InputDataConfig', 'OutputDataConfig', 'RoleArn', 'AutoMLJobConfig', 'CreationTime', 'EndTime', 'LastModifiedTime', 'BestCandidate', 'AutoMLJobStatus', 'AutoMLJobSecondaryStatus', 'GenerateCandidateDefinitionsOnly', 'AutoMLJobArtifacts', 'ResolvedAttributes', 'ResponseMetadata'])


In [126]:
while "BestCandidate" not in best_candidate_response:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    print("[INFO] Autopilot Job is generating BestCandidate. Please wait. ")
    print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate generated.")

[OK] BestCandidate generated.


In [127]:
print(json.dumps(best_candidate_response, indent=4, sort_keys=True, default=str))

{
    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:857283526476:automl-job/automl-dm-30-03-35-20",
    "AutoMLJobArtifacts": {
        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",
        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/sagemaker-automl-candidates/automl-dm-30-03-35-20-pr-1-927b84ee7c1b45949841948ad84488aa7395/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"
    },
    "AutoMLJobConfig": {
        "CompletionCriteria": {
            "MaxAutoMLJobRuntimeInSeconds": 5400,
            "MaxCandidates": 3,
            "MaxRuntimePerTrainingJobInSeconds": 900
        }
    },
    "AutoMLJobName": "automl-dm-30-03-35-20",
    "AutoMLJobSecondaryStatus": "Completed",
    "AutoMLJo

In [128]:
print(best_candidate.keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime', 'CandidateProperties'])


In [129]:
while "CandidateName" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateName. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate CandidateName generated.")

[OK] BestCandidate CandidateName generated.


In [130]:
while "FinalAutoMLJobObjectiveMetric" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate FinalAutoMLJobObjectiveMetric. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.")

[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.


In [131]:
best_candidate_identifier = best_candidate["CandidateName"]
print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate["FinalAutoMLJobObjectiveMetric"]["MetricName"])
print("Metric value: " + str(best_candidate["FinalAutoMLJobObjectiveMetric"]["Value"]))

Candidate name: automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc
Metric name: validation:accuracy
Metric value: 0.4800800085067749


In [132]:
print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))

{
    "CandidateName": "automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc",
    "CandidateProperties": {
        "CandidateArtifactLocations": {
            "Explainability": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/documentation/explainability/output",
            "ModelInsights": "s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/documentation/model_monitor/output"
        },
        "CandidateMetrics": [
            {
                "MetricName": "F1macro",
                "Set": "Validation",
                "StandardMetricName": "F1macro",
                "Value": 0.47442999482154846
            },
            {
                "MetricName": "PrecisionMacro",
                "Set": "Validation",
                "StandardMetricName": "PrecisionMacro",
                "Value": 0.4722200036048889
            },
            {
                "MetricName": "Accuracy",
                "Set": "Validation",
                "Stan

# Autopilot Jobs

In [133]:
while "CandidateSteps" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateSteps. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate CandidateSteps generated.")

[OK] BestCandidate CandidateSteps generated.


In [134]:
while "CandidateStepType" not in best_candidate["CandidateSteps"][0]:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateSteps CandidateStepType. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate CandidateSteps CandidateStepType generated.")

[OK] BestCandidate CandidateSteps CandidateStepType generated.


In [135]:
while "CandidateStepName" not in best_candidate["CandidateSteps"][0]:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate CandidateSteps CandidateStepName. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

best_candidate = best_candidate_response["BestCandidate"]
print("[OK] BestCandidate CandidateSteps CandidateStepName generated.")

[OK] BestCandidate CandidateSteps CandidateStepName generated.


In [136]:
best_candidate

{'CandidateName': 'automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy',
  'Value': 0.4800800085067749,
  'StandardMetricName': 'Accuracy'},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:857283526476:processing-job/automl-dm-30-03-35-20-db-1-48e7c029b9654b7a86c2751744ab05e66250',
   'CandidateStepName': 'automl-dm-30-03-35-20-db-1-48e7c029b9654b7a86c2751744ab05e66250'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:857283526476:training-job/automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91ac431e0ddec9d523d9',
   'CandidateStepName': 'automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91ac431e0ddec9d523d9'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:857283526476:transform-job/automl-dm-30-03-35-20

In [137]:
steps = []
for step in best_candidate["CandidateSteps"]:
    print("Candidate Step Type: {}".format(step["CandidateStepType"]))
    print("Candidate Step Name: {}".format(step["CandidateStepName"]))
    steps.append(step["CandidateStepName"])

Candidate Step Type: AWS::SageMaker::ProcessingJob
Candidate Step Name: automl-dm-30-03-35-20-db-1-48e7c029b9654b7a86c2751744ab05e66250
Candidate Step Type: AWS::SageMaker::TrainingJob
Candidate Step Name: automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91ac431e0ddec9d523d9
Candidate Step Type: AWS::SageMaker::TransformJob
Candidate Step Name: automl-dm-30-03-35-20-dpp2-rpb-1-b8add5e432b44f23a438a85a513d26
Candidate Step Type: AWS::SageMaker::TrainingJob
Candidate Step Name: automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc


In [138]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, steps[0]
        )
    )
)

In [139]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a></b>'.format(
            region, steps[1]
        )
    )
)

In [140]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/transform-jobs/{}">Transform Job</a></b>'.format(
            region, steps[2]
        )
    )
)

In [141]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review Best Candidate <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job (Tuning)</a></b>'.format(
            region, steps[3]
        )
    )
)

In [142]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review All <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">Processing Jobs</a></b>'.format(
            region
        )
    )
)

# See Containers and Models Within Inference Pipeline

In [143]:
while "InferenceContainers" not in best_candidate:
    best_candidate_response = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
    best_candidate = best_candidate_response["BestCandidate"]
    print("[INFO] Autopilot Job is generating BestCandidate InferenceContainers. Please wait. ")
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print("[OK] BestCandidate InferenceContainers generated.")

[OK] BestCandidate InferenceContainers generated.


In [144]:
best_candidate_containers = best_candidate["InferenceContainers"]

In [145]:
for container in best_candidate_containers:
    print(container["Image"])
    print(container["ModelDataUrl"])
    print("======================")

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3
s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/data-processor-models/automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91ac431e0ddec9d523d9/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3
s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/tuning/automl-dm--dpp2-xgb/automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3
s3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/data-processor-models/automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91ac431e0ddec9d523d9/output/model.tar.gz


# Update Containers To Show Predicted Label and Confidence Score

In [146]:
for container in best_candidate_containers:
    print(container["Environment"])
    print("======================")

{'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
{'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities'}
{'AUTOML_TRANSFORM_MODE': 'inverse-label-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_INPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,labels,probabilities', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}


In [147]:
best_candidate_containers[1]["Environment"].update({"SAGEMAKER_INFERENCE_OUTPUT": "predicted_label, probability"})
best_candidate_containers[2]["Environment"].update({"SAGEMAKER_INFERENCE_INPUT": "predicted_label, probability"})
best_candidate_containers[2]["Environment"].update({"SAGEMAKER_INFERENCE_OUTPUT": "predicted_label, probability"})

In [148]:
for container in best_candidate_containers:
    print(container["Environment"])
    print("======================")

{'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}
{'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,probabilities'}
{'AUTOML_TRANSFORM_MODE': 'inverse-label-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_INPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_label, probability', 'SAGEMAKER_INFERENCE_SUPPORTED': 'predicted_label,probability,labels,probabilities', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}


In [149]:
print(best_candidate["InferenceContainers"])

[{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:2.5-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/data-processor-models/automl-dm-30-03-35-20-dpp2-1-a02febbbfa594f91ac431e0ddec9d523d9/output/model.tar.gz', 'Environment': {'AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF': '1', 'AUTOML_TRANSFORM_MODE': 'feature-transform', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'application/x-recordio-protobuf', 'SAGEMAKER_PROGRAM': 'sagemaker_serve', 'SAGEMAKER_SUBMIT_DIRECTORY': '/opt/ml/model/code'}}, {'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.3-1-cpu-py3', 'ModelDataUrl': 's3://sagemaker-us-east-1-857283526476/models/autopilot/automl-dm-30-03-35-20/tuning/automl-dm--dpp2-xgb/automl-dm-30-03-35-206UAvnGVe6yJ-002-c9d4bccc/output/model.tar.gz', 'Environment': {'MAX_CONTENT_LENGTH': '20971520', 'SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT': 'text/csv', 'SAGEMAKER_INFERENCE_OUTPUT': 'predicted_

# Deploy the Model as a REST Endpoint
Batch transformations are also supported, but for now, we will use a REST Endpoint.

In [150]:
%store -r autopilot_model_wf

no stored variable or alias autopilot_model_wf


In [151]:
try:
    autopilot_model_wf
except NameError:
    timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
    autopilot_model_wf = "automl-dm-model-" + timestamp_suffix
    print("[OK] Created Autopilot Model Name: " + autopilot_model_wf)

[OK] Created Autopilot Model Name: automl-dm-model-30-04-47-47


In [152]:
%store autopilot_model_wf

Stored 'autopilot_model_wf' (str)


In [153]:
%store -r autopilot_model_arn_wf

no stored variable or alias autopilot_model_arn_wf


In [154]:
try:
    autopilot_model_arn_wf
except NameError:
    create_model_response = sm.create_model(
        Containers=best_candidate["InferenceContainers"], ModelName=autopilot_model_wf, ExecutionRoleArn=role
    )
    autopilot_model_arn_wf = create_model_response["ModelArn"]
    print("[OK] Created Autopilot Model: {}".format(autopilot_model_arn_wf))

[OK] Created Autopilot Model: arn:aws:sagemaker:us-east-1:857283526476:model/automl-dm-model-30-04-47-47


In [155]:
%store autopilot_model_arn_wf

Stored 'autopilot_model_arn_wf' (str)


# Define EndpointConfig Name

In [156]:
timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())
epc_name = "automl-dm-epc-" + timestamp_suffix

print(epc_name)

automl-dm-epc-30-04-47-49


# Define REST Endpoint Name for Autopilot Model

In [157]:
%store -r autopilot_endpoint_wf

no stored variable or alias autopilot_endpoint_wf


In [161]:
timestamp_suffix = strftime("%d-%H-%M-%S", gmtime())

try:
    autopilot_endpoint_wf
except NameError:
    autopilot_endpoint_wf = "automl-dm-ep-" + timestamp_suffix
    print("[OK] Created Autopilot Endpoint Name {}: ".format(autopilot_endpoint_wf))

In [162]:
variant_name = "automl-dm-variant-" + timestamp_suffix
print("[OK] Created Endpoint Variant Name {}: ".format(variant_name))

[OK] Created Endpoint Variant Name automl-dm-variant-30-04-48-18: 


In [163]:
%store autopilot_endpoint_wf

Stored 'autopilot_endpoint_wf' (str)


In [164]:
ep_config = sm.create_endpoint_config(
    EndpointConfigName=epc_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m5.large",
            "InitialInstanceCount": 1,
            "ModelName": autopilot_model_wf,
            "VariantName": variant_name,
        }
    ],
)

In [165]:
%store -r autopilot_endpoint_arn

no stored variable or alias autopilot_endpoint_arn


In [167]:
try:
    autopilot_endpoint_arn
except NameError:
    create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_wf, EndpointConfigName=epc_name)
    autopilot_endpoint_arn = create_endpoint_response["EndpointArn"]
    print(autopilot_endpoint_arn)

arn:aws:sagemaker:us-east-1:857283526476:endpoint/automl-dm-ep-30-04-47-49


In [168]:
%store autopilot_endpoint_arn

Stored 'autopilot_endpoint_arn' (str)


In [170]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST Endpoint</a></b>'.format(
            region, autopilot_endpoint_wf
        )
    )
)

# Store Variables

In [171]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-30-03-35-20'
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:857283526476:endpoint
autopilot_endpoint_wf                                 -> 'automl-dm-ep-30-04-47-49'
autopilot_model_arn_wf                                -> 'arn:aws:sagemaker:us-east-1:857283526476:model/au
autopilot_model_wf                                    -> 'automl-dm-model-30-04-47-47'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-857283526476/data/amazon
autopilot_train_s3_wf                                 -> 's3://sagemaker-us-east-1-857283526476/data/wildfi
balanced_bias_data_jsonlines_s3_uri                   -> 's3://sagemaker-us-east-1-857283526476/bias-detect
balanced_bias_data_s3_uri                             -> 's3://sagemaker-us-east-1-857283526476/bias-detect
bias_data_s3_uri                                      -> 's

# Release

In [172]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [173]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>